In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os 
import json
import numpy as np
import tqdm

# tourch imports
import os
import gc
import cv2
import json
import time

import numpy as np
import pandas as pd
from pathlib import Path
from keras.utils import to_categorical

import seaborn as sns
 
from matplotlib import colors
import matplotlib.pyplot as plt
 

import torch
T = torch.Tensor
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn.functional as F
import tensorflow as tf

Using TensorFlow backend.


In [2]:
#Data Directory for training data , evaluation data
 
training_path = 'C:/Users/Sai Karthik Yadav/ARC/data/training'
evaluation_path = 'C:/Users/Sai Karthik Yadav/ARC/data/evaluation'
test_path = 'C:/Users/Sai Karthik Yadav/ARC/data/test' 

In [3]:
training_files = [n for n in os.listdir(training_path)]
print(len(training_files))
evaluation_files = [n for n in os.listdir(evaluation_path)]
print(len(evaluation_files))

400
400


In [4]:
train_input_X = []
train_output_Y = []
for filenames in training_files:
    print(filenames)
    with open(training_path +'/'+filenames, 'r') as file_data:
        data = json.load(file_data)
        print(data)
        for train_sample in data['train']:
            train_input_X.append(np.array(train_sample['input']).shape)
            train_output_Y.append(np.array(train_sample['output']).shape)
            print(np.array(train_sample['input']).shape)
            print(np.array(train_sample['output']).shape)

007bbfb7.json
{'test': [{'input': [[7, 0, 7], [7, 0, 7], [7, 7, 0]], 'output': [[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]}], 'train': [{'input': [[0, 7, 7], [7, 7, 7], [0, 7, 7]], 'output': [[0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7]]}, {'input': [[4, 0, 4], [0, 0, 0], [0, 4, 0]], 'output': [[4, 0, 4, 0, 0, 0, 4, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 0, 0, 0, 0, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 0, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 4, 0, 0, 0, 0]]}, {'input'

{'train': [{'input': [[0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0], [0, 3, 0, 3, 0, 0], [0, 0, 3, 0, 3, 0], [0, 0, 0, 3, 0, 0], [0, 0, 0, 0, 0, 0]], 'output': [[0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0], [0, 3, 4, 3, 0, 0], [0, 0, 3, 4, 3, 0], [0, 0, 0, 3, 0, 0], [0, 0, 0, 0, 0, 0]]}, {'input': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 3, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 3, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0, 3, 0, 0, 0], [0, 0, 0, 0, 0, 3, 0, 3, 0, 0], [0, 0, 0, 3, 0, 3, 3, 0, 0, 0], [0, 0, 3, 3, 3, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'output': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 3, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 3, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0, 3, 0, 0, 0], [0, 0, 0, 0, 0, 3, 4, 3, 0, 0], [0, 0, 0, 3, 0, 3, 3, 0, 0, 0], [0, 0, 3, 3, 3, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}, {'input': [[0, 0, 0, 0, 0, 3, 0, 0, 0, 0], [0, 0, 0, 0, 3, 

(6, 3)
(9, 3)
(6, 3)
(9, 3)
025d127b.json
{'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 6, 6, 0, 0, 0, 0, 0], [0, 6, 0, 0, 6, 0, 0, 0, 0], [0, 0, 6, 0, 0, 6, 0, 0, 0], [0, 0, 0, 6, 0, 0, 6, 0, 0], [0, 0, 0, 0, 6, 6, 6, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 2, 2, 2, 0, 0, 0, 0], [0, 0, 2, 0, 0, 2, 0, 0, 0], [0, 0, 0, 2, 2, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'output': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 6, 6, 6, 0, 0, 0, 0], [0, 0, 6, 0, 0, 6, 0, 0, 0], [0, 0, 0, 6, 0, 0, 6, 0, 0], [0, 0, 0, 0, 6, 0, 6, 0, 0], [0, 0, 0, 0, 6, 6, 6, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 2, 2, 0, 0, 0], [0, 0, 0, 2, 0, 2, 0, 0, 0], [0, 0, 0, 2, 2, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]}, {'input': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 8, 8, 8, 8, 8, 0, 0, 0], [0, 8, 0, 0, 0, 0, 8, 0, 0], [0, 0, 8, 0,

{'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 8, 8, 8, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 8, 0, 8, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 8, 8, 8, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0,

(21, 21)
(21, 21)
(21, 21)
(21, 21)
(21, 21)
(21, 21)
0520fde7.json
{'train': [{'input': [[1, 0, 0, 5, 0, 1, 0], [0, 1, 0, 5, 1, 1, 1], [1, 0, 0, 5, 0, 0, 0]], 'output': [[0, 0, 0], [0, 2, 0], [0, 0, 0]]}, {'input': [[1, 1, 0, 5, 0, 1, 0], [0, 0, 1, 5, 1, 1, 1], [1, 1, 0, 5, 0, 1, 0]], 'output': [[0, 2, 0], [0, 0, 2], [0, 2, 0]]}, {'input': [[0, 0, 1, 5, 0, 0, 0], [1, 1, 0, 5, 1, 0, 1], [0, 1, 1, 5, 1, 0, 1]], 'output': [[0, 0, 0], [2, 0, 0], [0, 0, 2]]}], 'test': [{'input': [[1, 0, 1, 5, 1, 0, 1], [0, 1, 0, 5, 1, 0, 1], [1, 0, 1, 5, 0, 1, 0]], 'output': [[2, 0, 2], [0, 0, 0], [0, 0, 0]]}]}
(3, 7)
(3, 3)
(3, 7)
(3, 3)
(3, 7)
(3, 3)
05269061.json
{'train': [{'input': [[2, 8, 3, 0, 0, 0, 0], [8, 3, 0, 0, 0, 0, 0], [3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'output': [[2, 8, 3, 2, 8, 3, 2], [8, 3, 2, 8, 3, 2, 8], [3, 2, 8, 3, 2, 8, 3], [2, 8, 3, 2, 8, 3, 2], [8, 3, 2, 8, 3, 2, 8], [3, 2, 8, 3, 2, 8, 3], [2, 8, 3, 2, 

...

(14, 14)
(17, 14)
(17, 14)
(17, 16)
(17, 16)
(17, 16)
(17, 16)
f5b8619d.json
{'train': [{'input': [[2, 0, 0], [0, 0, 0], [0, 0, 2]], 'output': [[2, 0, 8, 2, 0, 8], [8, 0, 8, 8, 0, 8], [8, 0, 2, 8, 0, 2], [2, 0, 8, 2, 0, 8], [8, 0, 8, 8, 0, 8], [8, 0, 2, 8, 0, 2]]}, {'input': [[0, 5, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 5], [0, 0, 0, 0, 0, 0]], 'output': [[8, 5, 0, 0, 0, 8, 8, 5, 0, 0, 0, 8], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8], [5, 8, 0, 0, 0, 5, 5, 8, 0, 0, 0, 5], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8], [8, 5, 0, 0, 0, 8, 8, 5, 0, 0, 0, 8], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8], [5, 8, 0, 0, 0, 5, 5, 8, 0, 0, 0, 5], [8, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 8]]}, {'input': [[0, 4], [0, 0]], 'output': [[0, 4, 0, 4], [0, 8, 0, 8], [0, 4, 0, 4], [0, 8, 0, 8]]}], 'test': [{'input': [[0, 0, 3, 0], [

{'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 3, 3, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 1, 0, 4, 4, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 4, 4, 0, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 2, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'output': [[4], [2], [3]]}, {'input': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 6, 0, 8, 8, 0, 8, 8, 0, 8, 8, 0, 8, 8, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 8, 8, 0, 8, 8, 0, 2, 2, 0, 6, 6, 0, 8, 8, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 8, 8, 0, 1, 1, 0, 8, 8, 0, 8, 8, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 8, 8, 0, 1, 1, 0, 8, 8, 0, 8, 8, 0, 8, 8, 0], [0, 0

{'train': [{'input': [[2, 1, 2, 1, 2], [1, 1, 1, 1, 1], [2, 1, 2, 1, 2], [0, 0, 1, 1, 1], [0, 0, 2, 1, 2]], 'output': [[1, 1], [2, 1]]}, {'input': [[8, 6, 0, 6], [6, 8, 6, 8], [8, 6, 8, 6], [6, 8, 6, 8]], 'output': [[8]]}, {'input': [[2, 2, 5, 2, 2, 5, 2], [2, 2, 5, 2, 2, 5, 2], [5, 5, 5, 5, 5, 5, 5], [2, 2, 5, 2, 2, 5, 2], [2, 2, 5, 2, 2, 5, 2], [5, 5, 5, 5, 5, 0, 0], [2, 2, 5, 2, 2, 0, 0]], 'output': [[5, 5], [5, 2]]}], 'test': [{'input': [[8, 1, 8, 8, 0, 0, 0], [1, 8, 8, 1, 0, 0, 0], [8, 8, 1, 8, 0, 0, 0], [8, 1, 8, 8, 1, 8, 8], [1, 8, 8, 1, 8, 8, 1], [8, 8, 1, 8, 8, 1, 8], [8, 1, 8, 8, 1, 8, 8]], 'output': [[1, 8, 8], [8, 8, 1], [8, 1, 8]]}]}
(5, 5)
(2, 2)
(4, 4)
(1, 1)
(7, 7)
(2, 2)
fafffa47.json
{'test': [{'input': [[9, 0, 9], [0, 0, 9], [9, 0, 9], [0, 1, 1], [0, 1, 0], [1, 0, 0]], 'output': [[0, 0, 0], [2, 0, 0], [0, 2, 0]]}], 'train': [{'input': [[0, 9, 9], [0, 9, 9], [9, 9, 9], [0, 1, 0], [0, 0, 1], [1, 1, 1]], 'output': [[2, 0, 0], [2, 0, 0], [0, 0, 0]]}, {'input': [[9, 0, 9]

{'train': [{'input': [[1, 0, 7, 0, 0]], 'output': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 7], [0, 0, 0, 0, 0, 0, 1, 0, 7, 0], [0, 0, 0, 0, 0, 1, 0, 7, 0, 0], [0, 0, 0, 0, 1, 0, 7, 0, 0, 0], [0, 0, 0, 1, 0, 7, 0, 0, 0, 0], [0, 0, 1, 0, 7, 0, 0, 0, 0, 0], [0, 1, 0, 7, 0, 0, 0, 0, 0, 0], [1, 0, 7, 0, 0, 0, 0, 0, 0, 0]]}, {'input': [[0, 0, 2, 0, 0]], 'output': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 2], [0, 0, 0, 2, 0], [0, 0, 2, 0, 0]]}, {'input': [[4, 0, 6, 0, 8]], 'output': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 0, 8], [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 0, 8, 0], [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 0, 8, 0, 0], [0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 0, 8, 0, 0, 0], [0, 0, 0, 0, 0, 0, 4, 0, 6, 0, 8, 0, 0, 0, 0], [0, 0, 0, 0, 0, 4, 0, 6, 0, 8, 0, 0, 

{'test': [{'input': [[4, 4, 4, 0, 4, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 3, 3, 0, 0, 4, 0, 4, 4, 4], [4, 4, 4, 4, 0, 4, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 4, 0, 4, 4, 4, 4], [4, 4, 0, 4, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 4, 0, 4, 4], [0, 4, 4, 0, 4, 4, 3, 3, 0, 0, 3, 3, 3, 3, 0, 0, 3, 3, 4, 4, 0, 4, 4, 0], [4, 0, 0, 4, 4, 4, 0, 0, 3, 3, 0, 3, 3, 0, 3, 3, 0, 0, 4, 4, 4, 0, 0, 4], [0, 4, 0, 4, 4, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 4, 4, 0, 4, 0], [0, 3, 3, 3, 0, 0, 8, 8, 8, 1, 1, 1, 1, 1, 8, 8, 8, 8, 0, 0, 3, 3, 3, 0], [3, 3, 3, 3, 0, 3, 8, 8, 8, 1, 1, 1, 1, 1, 0, 8, 8, 8, 3, 0, 3, 3, 3, 3], [3, 3, 0, 0, 3, 3, 8, 8, 8, 1, 1, 1, 1, 1, 0, 8, 8, 8, 3, 3, 0, 0, 3, 3], [3, 3, 0, 0, 3, 3, 8, 0, 0, 1, 1, 1, 1, 1, 8, 0, 0, 8, 3, 3, 0, 0, 3, 3], [0, 0, 3, 3, 0, 3, 8, 0, 8, 1, 1, 1, 1, 1, 8, 8, 0, 8, 3, 0, 3, 3, 0, 0], [0, 3, 3, 3, 3, 3, 8, 8, 0, 8, 8, 8, 8, 8, 8, 0, 8, 8, 3, 3, 3, 3, 3, 0], [0, 3, 3, 3, 3, 3, 8, 8, 0, 8, 8, 8, 8, 8, 8, 0, 8, 8, 3, 3, 3, 3, 3, 0], [0, 0, 3, 3, 0, 3

In [5]:
tensor_input = torch.FloatTensor(train_input_X)
tensor_output = torch.FloatTensor(train_output_Y)
#output_Y
print(tensor_input.dim())
print(tensor_input.shape)

2
torch.Size([1301, 2])


In [6]:
train_ds = TensorDataset(tensor_input,tensor_output)
train_ds[0:5]

(tensor([[3., 3.],
         [3., 3.],
         [3., 3.],
         [3., 3.],
         [3., 3.]]), tensor([[9., 9.],
         [9., 9.],
         [9., 9.],
         [9., 9.],
         [9., 9.]]))

In [7]:
#define data loadder
batch_size = 4
train_dl = DataLoader(train_ds,batch_size,shuffle=True)
next(iter(train_dl))

[tensor([[10., 10.],
         [16., 18.],
         [10.,  3.],
         [ 3.,  3.]]), tensor([[ 3.,  3.],
         [ 9.,  9.],
         [10.,  3.],
         [ 3.,  3.]])]

In [8]:
#Define the model, nn.Linear takes care of intializng the weights
model = nn.Linear(2,2) #3 inputs and 2 outputs
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3613,  0.4852],
        [-0.6711, -0.6767]], requires_grad=True)
Parameter containing:
tensor([-0.6694, -0.3014], requires_grad=True)


In [9]:
opt = torch.optim.SGD(model.parameters(),lr=1e-4)

In [10]:
#Use the builtin loss function mse_loss
loss_fn = F.mse_loss

In [11]:
loss = loss_fn(model(tensor_input),tensor_output)
print(loss)

tensor(354.5040, grad_fn=<MseLossBackward>)


In [12]:
#Define a utility functin to train the model
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            #Generate Predictions
            pred = model(xb)
            loss = loss_fn(pred,yb)
            #Perform Gradient Descent
            loss.backward()
            opt.step()
            #set the gradients to zero
            opt.zero_grad()
    print('Training Loss: ', loss_fn(model(tensor_input),tensor_output))

**One shot training **

In [ ]:
#Train the model for 100 epochs
fit(5000,model,loss_fn,opt)

In [ ]:
#Generate predictions
preds = model(tensor_input).detach().numpy()
preds 

In [ ]:
#compare with targets
tensor_output

EVALUATION 

In [ ]:
def get_diff_percentage(current, previous):
    if current == previous:
        return 100.0
    try:
        return (abs(current - previous) / previous) * 100.0
    except ZeroDivisionError:
        return 0

In [ ]:
score = 0
total_files = 0
score_with_less_diff = 0
for filenames in training_files:     
    with open(training_path +'/'+filenames, 'r') as file_data:
        data = json.load(file_data)         
        for train_sample in data['train']:
             total_files = total_files + 1 
             ml_predict = model(torch.FloatTensor(np.array(train_sample['input']).shape)).detach().numpy()
             ex_output = np.array(train_sample['output']).shape   
             print('File Name: ' + filenames)
             print('Input shapes:\t\t', np.array(train_sample['input']).shape)
             print('Predicted shapes:\t', ml_predict)
             print('Expected shapes:\t', ex_output)
             
             sc1 = get_diff_percentage(round(ml_predict[0], 0),ex_output[0]) 
             sc2 = get_diff_percentage(round(ml_predict[0], 0),ex_output[0]) 
             if(round(ml_predict[0], 0) == ex_output[0] or round(ml_predict[1], 0) ==  ex_output[1]):
                    score = score + 1
             if(sc1 < 50 and sc2 < 50):
                score_with_less_diff = score_with_less_diff +1 
                
             
          
             
             

In [ ]:
print('Score ' + str(score))  
print('Score With less difference with expected - avg less than 50 % -> ' + str(score_with_less_diff))  
print('Total files ' + str(total_files))

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = ['Correctly Predicted','Wrongly Predicted']
students = [32,368]
ax.set_title('Convolution Neural Network Prediction - Validation')
ax.set_ylabel('Total Taks')
barlist=ax.bar(langs,students)
barlist[0].set_color('b')
barlist[1].set_color('r')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.axis('equal')
langs = ['Correctly predicted tasks', 'Wrongly Predicted tasks']
students = [32,368]
ax.pie(students, labels = langs,autopct='%1.2f%%')
ax.set_title('Convolution Neural Network Shape Prediction - Accuracy')
plt.show()